# Cascade NAS Tutorial: Neuromorphic Auditory Processing with OKAERTool

This notebook provides a comprehensive tutorial on using the **Cascade Neuromorphic Auditory Sensor (NAS)** with the **OKAERTool** platform. You will learn how to configure, monitor, and analyze spike events from a bio-inspired auditory sensor deployed on FPGA hardware.

## What is the Cascade NAS?

The Cascade NAS is a neuromorphic auditory sensor that mimics the structure and function of the biological cochlea. It processes audio signals in real-time and outputs Address-Event Representation (AER) spike streams. The cascade architecture features:

- **64 frequency channels** arranged in a cascade topology
- **Bio-inspired bandpass filtering** mimicking cochlear mechanics
- **Spike-based output** for efficient neuromorphic processing
- **Real-time operation** suitable for edge computing applications

## What is OKAERTool?

**OKAERTool** ([https://github.com/RTC-research-group/okaertool](https://github.com/RTC-research-group/okaertool)) is an open-source hardware platform designed for deploying and testing AER-based neuromorphic systems on FPGA. It provides:

- **Hardware IP blocks** for AER event management
- **Python interface (pyOKAERTool)** for easy configuration and monitoring
- **Integration with pyNAVIS** for spike visualization
- **Support for multiple neuromorphic sensors** including NAS

This tutorial will guide you through the complete workflow from initialization to advanced event analysis.

## 1. Import Required Packages and Initialize OKAERTool

First, we need to import the necessary libraries and initialize the OKAERTool hardware platform. We'll also configure the pyNAVIS settings for spike visualization.

In [ ]:
import sys
import os

# Add repository root to sys.path for importing pyOKAERTool
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if os.path.isdir(os.path.join(repo_root, 'pyOKAERTool')) and repo_root not in sys.path:
    sys.path.insert(0, repo_root)
else:
    # Fallback: walk up parent directories to find pyOKAERTool
    p = os.path.abspath(os.getcwd())
    for _ in range(4):
        if os.path.isdir(os.path.join(p, 'pyOKAERTool')):
            if p not in sys.path:
                sys.path.insert(0, p)
            break
        p = os.path.dirname(p)

import pyOKAERTool as okt
from pyNAVIS import *
import numpy as np

# Define bitfile path for the FPGA configuration
bitfile_path = '../../bitfiles/CNAS_okaertool_XEM6310_config_0_rst.bit'

# Validate bitfile existence
if bitfile_path is not None and not os.path.exists(bitfile_path):
    print(f"ERROR: Bitfile not found at: {bitfile_path}")
    sys.exit(1)

# Create and initialize OKAERTool instance
print("Initializing OKAERTool...")
okaer = okt.Okaertool(bit_file=bitfile_path)
okaer.init()
print("✓ OKAERTool initialized successfully")

# Configure pyNAVIS settings for visualization
# Parameters:
#   - num_channels: 64 frequency channels in the Cascade NAS
#   - mono_stereo: 1 for stereo (left/right cochlea)
#   - on_off_both: 1 for both ON and OFF events
#   - address_size: 4 bytes for address representation
#   - ts_tick: 0.01 ms timestamp resolution
#   - bin_size: 10000 for histogram binning
settings = MainSettings(
    num_channels=64, 
    mono_stereo=1, 
    on_off_both=1, 
    address_size=4, 
    ts_tick=0.01, 
    bin_size=10000
)
print("✓ PyNAVIS settings configured")

## 2. Define Configuration Parameters

The Cascade NAS requires specific configuration parameters to operate correctly. These parameters control both the digital audio interface and the cochlear filter bank characteristics.

### I2S2Spikes Parameters

The **I2S2Spikes** module converts digital I2S audio signals into spike events. The parameter controls the sensitivity and thresholding behavior.

### CASCADE_FILTER Parameters

Each of the 64 cochlear channels requires 4 configuration parameters:

1. **FREQ_DIV** - Frequency divider determining the center frequency of the channel
2. **SPIKES_DIV_FB** - Spike divider for the feedback path (controls resonance)
3. **SPIKES_DIV_OUT** - Spike divider for the output (controls firing rate)
4. **SPIKES_DIV_BPF** - Spike divider for the bandpass filter (controls bandwidth)

These parameters define the frequency selectivity and temporal dynamics of each cochlear filter, arranged from high to low frequencies to mimic biological cochlear organization.

In [ ]:
# I2S2Spikes configuration parameter
# Controls the digital audio interface conversion to spikes
I2S2Spikes_DEFAULT_parameter = [0x000F]

# CASCADE_FILTER configuration parameters
# 64 filters × 4 parameters = 256 values
# Each group of 4 values configures one cochlear channel
CASCADE_FILTER_DEFAULT_parameter = [
    # Filter 1 (highest frequency)
    0x04, 0x77B4, 0x77B4, 0x2025, 
    # Filter 2
    0x04, 0x6B1C, 0x6B1C, 0x2025,
    # Filter 3
    0x02, 0x7303, 0x7303, 0x2025, 
    # Filter 4
    0x02, 0x66E9, 0x66E9, 0x2025, 
    # Filter 5
    0x03, 0x7AC8, 0x7AC8, 0x2025, 
    # Filter 6
    0x03, 0x6DDD, 0x6DDD, 0x2025, 
    # Filter 7
    0x04, 0x7AE1, 0x7AE1, 0x2025, 
    # Filter 8
    0x04, 0x6DF4, 0x6DF4, 0x2025, 
    # Filter 9
    0x02, 0x7610, 0x7610, 0x2025, 
    # Filter 10
    0x02, 0x69A4, 0x69A4, 0x2025, 
    # Filter 11
    0x03, 0x7E0A, 0x7E0A, 0x2025, 
    # Filter 12
    0x03, 0x70C7, 0x70C7, 0x2025, 
    # Filter 13
    0x04, 0x7E24, 0x7E24, 0x2025, 
    # Filter 14
    0x04, 0x70DF, 0x70DF, 0x2025, 
    # Filter 15
    0x02, 0x7932, 0x7932, 0x2025, 
    # Filter 16
    0x02, 0x6C72, 0x6C72, 0x2025, 
    # Filter 17
    0x02, 0x6109, 0x6109, 0x2025, 
    # Filter 18
    0x03, 0x73C5, 0x73C5, 0x2025, 
    # Filter 19
    0x03, 0x6797, 0x6797, 0x2025, 
    # Filter 20
    0x04, 0x73DD, 0x73DD, 0x2025, 
    # Filter 21
    0x02, 0x7C69, 0x7C69, 0x2025, 
    # Filter 22
    0x02, 0x6F52, 0x6F52, 0x2025, 
    # Filter 23
    0x02, 0x639C, 0x639C, 0x2025, 
    # Filter 24
    0x03, 0x76D8, 0x76D8, 0x2025, 
    # Filter 25
    0x03, 0x6A57, 0x6A57, 0x2025, 
    # Filter 26
    0x04, 0x76F1, 0x76F1, 0x2025, 
    # Filter 27
    0x02, 0x7FB6, 0x7FB6, 0x2025, 
    # Filter 28
    0x02, 0x7247, 0x7247, 0x2025, 
    # Filter 29
    0x02, 0x6641, 0x6641, 0x2025, 
    # Filter 30
    0x03, 0x79FF, 0x79FF, 0x2025, 
    # Filter 31
    0x03, 0x6D29, 0x6D29, 0x2025, 
    # Filter 32
    0x04, 0x7A19, 0x7A19, 0x2025, 
    # Filter 33 (middle frequencies)
    0x04, 0x6D40, 0x6D40, 0x2025, 
    # Filter 34
    0x02, 0x754F, 0x754F, 0x2025, 
    # Filter 35
    0x02, 0x68F7, 0x68F7, 0x2025, 
    # Filter 36
    0x03, 0x7D3C, 0x7D3C, 0x2025, 
    # Filter 37
    0x03, 0x700F, 0x700F, 0x2025, 
    # Filter 38
    0x04, 0x7D56, 0x7D56, 0x2025, 
    # Filter 39
    0x04, 0x7026, 0x7026, 0x2025, 
    # Filter 40
    0x02, 0x786C, 0x786C, 0x2025, 
    # Filter 41
    0x02, 0x6BC1, 0x6BC1, 0x2025, 
    # Filter 42
    0x02, 0x606A, 0x606A, 0x2025, 
    # Filter 43
    0x03, 0x7308, 0x7308, 0x2025, 
    # Filter 44
    0x03, 0x66EE, 0x66EE, 0x2025, 
    # Filter 45
    0x04, 0x7320, 0x7320, 0x2025, 
    # Filter 46
    0x02, 0x7B9E, 0x7B9E, 0x2025, 
    # Filter 47
    0x02, 0x6E9C, 0x6E9C, 0x2025, 
    # Filter 48
    0x02, 0x62F9, 0x62F9, 0x2025, 
    # Filter 49
    0x03, 0x7615, 0x7615, 0x2025, 
    # Filter 50
    0x03, 0x69A9, 0x69A9, 0x2025, 
    # Filter 51
    0x04, 0x762E, 0x762E, 0x2025, 
    # Filter 52
    0x02, 0x7EE6, 0x7EE6, 0x2025, 
    # Filter 53
    0x02, 0x718C, 0x718C, 0x2025, 
    # Filter 54
    0x02, 0x659A, 0x659A, 0x2025, 
    # Filter 55
    0x03, 0x7937, 0x7937, 0x2025, 
    # Filter 56
    0x03, 0x6C77, 0x6C77, 0x2025, 
    # Filter 57
    0x04, 0x7951, 0x7951, 0x2025, 
    # Filter 58
    0x04, 0x6C8E, 0x6C8E, 0x2025, 
    # Filter 59
    0x02, 0x748F, 0x748F, 0x2025, 
    # Filter 60
    0x02, 0x684C, 0x684C, 0x2025, 
    # Filter 61
    0x03, 0x7C6F, 0x7C6F, 0x2025, 
    # Filter 62
    0x03, 0x6F57, 0x6F57, 0x2025,
    # Filter 63
    0x04, 0x7C89, 0x7C89, 0x2025, 
    # Filter 64 (lowest frequency)
    0x04, 0x6F6F, 0x6F6F, 0x2025, 
    0x02, 0x77A7, 0x77A7, 0x2025
]

print(f"Configuration parameters loaded:")
print(f"  - I2S2Spikes: {len(I2S2Spikes_DEFAULT_parameter)} parameters")
print(f"  - CASCADE_FILTER: {len(CASCADE_FILTER_DEFAULT_parameter)} parameters ({len(CASCADE_FILTER_DEFAULT_parameter)//4} filters)")

## 3. Configure the Cascade NAS

Now we'll program the FPGA with our configuration parameters. This involves:

1. **Resetting the board** to ensure a clean state
2. **Configuring I2S2Spikes** module for audio signal conversion
3. **Configuring left cochlea** cascade filters (64 channels)
4. **Configuring right cochlea** cascade filters (64 channels)

Each configuration writes parameter values to specific register addresses in the FPGA.

In [ ]:
# Reset the OKAERTool board
print("Resetting OKAERTool board...")
okaer.reset_board()
print("✓ Board reset complete")

# Configure I2S2Spikes module
# Register address: 0x08
register_address = 0x08
okaer.logger.info("Configuring I2S2Spikes module")
for value in I2S2Spikes_DEFAULT_parameter:
    okaer.set_config('port_a', register_address, value)
print("✓ I2S2Spikes configured")

# Configure CASCADE filters for LEFT cochlea
# Starting register address: 0x09
register_address = 0x09
okaer.logger.info("Configuring LEFT cochlea cascade filters")
for value in CASCADE_FILTER_DEFAULT_parameter:
    okaer.set_config('port_a', register_address, value)
    register_address += 1
print(f"✓ Left cochlea configured (64 filters)")

# Configure CASCADE filters for RIGHT cochlea
# Starting register address: 0x010D (269 in decimal)
register_address = 0x010D
okaer.logger.info("Configuring RIGHT cochlea cascade filters")
for value in CASCADE_FILTER_DEFAULT_parameter:
    okaer.set_config('port_a', register_address, value)
    register_address += 1
print(f"✓ Right cochlea configured (64 filters)")

print("\n" + "="*50)
print("CASCADE NAS CONFIGURATION COMPLETE")
print("="*50)

## 4. Monitor NAS Output Events

The OKAERTool provides flexible monitoring capabilities for capturing spike events from the NAS. You can monitor events based on:

- **Duration**: Capture events for a specific time period (e.g., 0.5 seconds)
- **Spike count**: Capture until a maximum number of spikes is reached
- **Both constraints**: Stop when either condition is met

Let's demonstrate all three monitoring modes.

### 4.1 Monitor by Duration

This is the most common monitoring mode - capture all events that occur during a specified time window.

In [ ]:
# Monitoring configuration
MAX_INPUTS = 3  # Number of input channels to monitor
INPUTS = ['port_a']  # Monitor port A (Cascade NAS)
DURATION = 0.5  # Monitoring duration in seconds

print(f"Monitoring for {DURATION} seconds...")
print("Waiting for spike events...")

# Monitor for a specific duration
spikes = okaer.monitor(inputs=INPUTS, duration=DURATION)

# Validate that spikes were captured
if spikes is None:
    okaer.logger.error("No spikes were recorded. Check audio input!")
    sys.exit(1)

# Print spike count for each input
print("\n" + "="*50)
print("MONITORING RESULTS (Duration mode)")
print("="*50)
for i in range(MAX_INPUTS):
    num_spikes = spikes[i].get_num_spikes()
    print(f"Input {i}: {num_spikes:,} spikes")
    if num_spikes > 0:
        print(f"  ├─ Spike rate: {num_spikes/DURATION:.1f} spikes/second")
print("="*50)

### 4.2 Monitor by Spike Count

Capture events until a specific number of spikes is reached, useful when you need a fixed amount of data regardless of time.

In [ ]:
MAX_SPIKES = 10000  # Maximum spikes to capture

print(f"Monitoring until {MAX_SPIKES:,} spikes are captured...")
print("Waiting for spike events...")

# Monitor for a specific number of spikes
spikes_count_mode = okaer.monitor(inputs=INPUTS, max_spikes=MAX_SPIKES)

if spikes_count_mode is None:
    okaer.logger.error("No spikes were recorded.")
else:
    print("\n" + "="*50)
    print("MONITORING RESULTS (Spike count mode)")
    print("="*50)
    for i in range(MAX_INPUTS):
        num_spikes = spikes_count_mode[i].get_num_spikes()
        print(f"Input {i}: {num_spikes:,} spikes")
    print("="*50)

### 4.3 Monitor with Both Constraints

Stop monitoring when EITHER the duration limit OR the spike count limit is reached - whichever comes first.

In [ ]:
DURATION_COMBINED = 2.0  # Maximum duration
MAX_SPIKES_COMBINED = 50000  # Maximum spikes

print(f"Monitoring with dual constraints:")
print(f"  - Duration limit: {DURATION_COMBINED} seconds")
print(f"  - Spike limit: {MAX_SPIKES_COMBINED:,} spikes")
print("Will stop when EITHER limit is reached...")

# Monitor with both constraints
spikes_combined = okaer.monitor(
    inputs=INPUTS, 
    max_spikes=MAX_SPIKES_COMBINED, 
    duration=DURATION_COMBINED
)

if spikes_combined is None:
    okaer.logger.error("No spikes were recorded.")
else:
    print("\n" + "="*50)
    print("MONITORING RESULTS (Combined constraints)")
    print("="*50)
    for i in range(MAX_INPUTS):
        num_spikes = spikes_combined[i].get_num_spikes()
        print(f"Input {i}: {num_spikes:,} spikes")
    print("="*50)

## 5. Visualize Spike Data

Now let's create visualizations using the captured spike data. We'll use **pyNAVIS** to generate four types of plots that help us understand the NAS response.

First, we create `SpikesFile` objects for all inputs that recorded spikes.

In [ ]:
# Create pyNAVIS SpikesFile objects for inputs with recorded spikes
okaer.logger.info("Creating spike files for visualization")
spike_files = []

for i in range(MAX_INPUTS):
    if spikes[i].get_num_spikes() > 0:
        spike_files.append(
            SpikesFile(
                addresses=spikes[i].addresses, 
                timestamps=spikes[i].timestamps
            )
        )
        print(f"✓ SpikesFile created for input {i}")

print(f"\nTotal spike files created: {len(spike_files)}")

### 5.1 Spikegram (Raster Plot)

The **spikegram** (also called raster plot) displays individual spike events as dots. Each row represents one of the 64 frequency channels, and each dot represents a spike event at a specific time.

**Interpretation:**
- **Horizontal axis**: Time progression
- **Vertical axis**: Frequency channel (0-63, high to low frequency)
- **Each dot**: A single spike event
- **Patterns**: Horizontal lines indicate sustained activity in specific frequency bands
- **Density**: More dots = higher neural activity

In [ ]:
# Generate spikegram for all inputs
for i in range(len(spike_files)):
    okaer.logger.info(f"Plotting spikegram for input {INPUTS[i]}")
    Plots.spikegram(spike_files[i], settings)

### 5.2 Sonogram (Frequency-Time Representation)

The **sonogram** provides a heat map view of spike activity, showing energy distribution across frequency channels over time.

**Interpretation:**
- **Horizontal axis**: Time progression
- **Vertical axis**: Frequency channel (cochlear position)
- **Color intensity**: Amount of spike activity (bright = high activity)
- **Patterns**: Bright regions indicate strong acoustic energy at those frequencies
- **Temporal evolution**: Shows how frequency content changes over time

This visualization is similar to a traditional spectrogram but based on neuromorphic spike events.

In [ ]:
# Generate sonogram for all inputs
for i in range(len(spike_files)):
    okaer.logger.info(f"Plotting sonogram for input {INPUTS[i]}")
    Plots.sonogram(spike_files[i], settings)

### 5.3 Histogram (Spike Count Distribution)

The **histogram** shows the total number of spikes per frequency channel across the entire recording period.

**Interpretation:**
- **Horizontal axis**: Frequency channel number (0-63)
- **Vertical axis**: Total spike count
- **Height of bars**: Indicates which channels were most active
- **Distribution shape**: Reveals the spectral content of the audio signal
- **Peaks**: Channels with highest activity correspond to dominant frequencies in the input

This plot helps identify which frequency ranges are most important in the input signal.

In [ ]:
# Generate histogram for all inputs
for i in range(len(spike_files)):
    okaer.logger.info(f"Plotting histogram for input {INPUTS[i]}")
    Plots.histogram(spike_files[i], settings)

### 5.4 Average Activity (Temporal Firing Rate)

The **average activity** plot shows how the mean firing rate across all channels evolves over time.

**Interpretation:**
- **Horizontal axis**: Time progression
- **Vertical axis**: Average spike rate (spikes per time bin)
- **Curve height**: Overall level of neural activity
- **Peaks**: Moments when the NAS is responding strongly to the input
- **Temporal dynamics**: Shows onset, sustained activity, and offset of the response

This visualization is useful for detecting transient events and understanding the temporal structure of the auditory input.

In [ ]:
# Generate average activity plot for all inputs
for i in range(len(spike_files)):
    okaer.logger.info(f"Plotting average activity for input {INPUTS[i]}")
    Plots.average_activity(spike_files[i], settings)

## 6. Process and Analyze Captured Events - Peak Activity Detection

Beyond visualization, we can perform computational analysis on the spike data. Let's implement an algorithm to **detect temporal windows with peak activity**.

This analysis identifies the time period(s) when the NAS is most active, which could correspond to important acoustic events like speech onset, musical notes, or transient sounds.

### Algorithm Overview:

1. Define a sliding time window (e.g., 50 milliseconds)
2. Slide this window across the recording
3. Count spikes in each window position
4. Identify the window with maximum activity
5. Analyze the channel distribution during that peak

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def detect_peak_activity_window(spike_file, window_size_ms=50, settings=None):
    """
    Detect the temporal window with highest spike activity.
    
    Parameters:
    -----------
    spike_file : SpikesFile
        PyNAVIS spike file object
    window_size_ms : float
        Size of sliding window in milliseconds
    settings : MainSettings
        PyNAVIS settings for timestamp conversion
        
    Returns:
    --------
    dict : Dictionary containing peak window information
    """
    timestamps = spike_file.timestamps
    addresses = spike_file.addresses
    
    if len(timestamps) == 0:
        print("No spikes to analyze!")
        return None
    
    # Convert window size to timestamp units
    ts_tick = settings.ts_tick if settings else 0.01  # default 0.01 ms
    window_size = window_size_ms / ts_tick
    
    # Get time range
    t_min = timestamps.min()
    t_max = timestamps.max()
    total_duration = t_max - t_min
    
    # Create sliding windows
    window_step = window_size / 4  # 75% overlap for smooth detection
    num_windows = int((total_duration - window_size) / window_step) + 1
    
    print(f"\nAnalyzing spike activity...")
    print(f"  - Total spikes: {len(timestamps):,}")
    print(f"  - Recording duration: {total_duration * ts_tick:.2f} ms")
    print(f"  - Window size: {window_size_ms} ms")
    print(f"  - Number of windows: {num_windows}")
    
    # Count spikes in each window
    spike_counts = []
    window_starts = []
    
    for i in range(num_windows):
        window_start = t_min + i * window_step
        window_end = window_start + window_size
        
        # Count spikes in this window
        spikes_in_window = np.sum((timestamps >= window_start) & (timestamps < window_end))
        spike_counts.append(spikes_in_window)
        window_starts.append(window_start)
    
    spike_counts = np.array(spike_counts)
    window_starts = np.array(window_starts)
    
    # Find peak window
    peak_idx = np.argmax(spike_counts)
    peak_start = window_starts[peak_idx]
    peak_end = peak_start + window_size
    peak_count = spike_counts[peak_idx]
    
    # Get spikes and channels in peak window
    peak_mask = (timestamps >= peak_start) & (timestamps < peak_end)
    peak_addresses = addresses[peak_mask]
    peak_timestamps = timestamps[peak_mask]
    
    # Analyze channel distribution in peak window
    channel_counts = np.bincount(peak_addresses, minlength=64)
    most_active_channels = np.argsort(channel_counts)[-5:][::-1]  # Top 5 channels
    
    print(f"\n{'='*60}")
    print(f"PEAK ACTIVITY WINDOW DETECTED")
    print(f"{'='*60}")
    print(f"  Start time: {peak_start * ts_tick:.2f} ms")
    print(f"  End time: {peak_end * ts_tick:.2f} ms")
    print(f"  Spikes in window: {peak_count:,}")
    print(f"  Spike rate: {peak_count / window_size_ms * 1000:.1f} spikes/second")
    print(f"\n  Top 5 most active channels during peak:")
    for rank, ch in enumerate(most_active_channels, 1):
        print(f"    {rank}. Channel {ch}: {channel_counts[ch]} spikes")
    print(f"{'='*60}")
    
    # Visualization
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    # Plot 1: Spike rate over time with peak highlighted
    ax1 = axes[0]
    time_axis = window_starts * ts_tick
    ax1.plot(time_axis, spike_counts, 'b-', linewidth=1.5, label='Spike count per window')
    ax1.axvspan(peak_start * ts_tick, peak_end * ts_tick, alpha=0.3, color='red', 
                label=f'Peak window ({peak_count} spikes)')
    ax1.axhline(y=np.mean(spike_counts), color='gray', linestyle='--', alpha=0.5, label='Mean activity')
    ax1.set_xlabel('Time (ms)', fontsize=12)
    ax1.set_ylabel('Spike Count', fontsize=12)
    ax1.set_title('Temporal Activity Profile with Peak Detection', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Channel distribution during peak window
    ax2 = axes[1]
    ax2.bar(range(64), channel_counts, color='steelblue', edgecolor='black', linewidth=0.5)
    ax2.bar(most_active_channels, channel_counts[most_active_channels], 
            color='red', edgecolor='black', linewidth=0.5, label='Top 5 channels')
    ax2.set_xlabel('Frequency Channel', fontsize=12)
    ax2.set_ylabel('Spike Count in Peak Window', fontsize=12)
    ax2.set_title('Frequency Distribution During Peak Activity', fontsize=14, fontweight='bold')
    ax2.set_xlim(-1, 64)
    ax2.legend()
    ax2.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    return {
        'peak_start': peak_start * ts_tick,
        'peak_end': peak_end * ts_tick,
        'peak_count': peak_count,
        'most_active_channels': most_active_channels,
        'channel_counts': channel_counts,
        'all_window_counts': spike_counts,
        'window_times': window_starts * ts_tick
    }

# Run peak detection analysis on the first spike file
if len(spike_files) > 0:
    peak_info = detect_peak_activity_window(spike_files[0], window_size_ms=50, settings=settings)
else:
    print("No spike data available for analysis!")

## 7. Process and Analyze Captured Events - Channel Response Analysis

Another important analysis is understanding **which frequency channels respond most strongly** to the input. This reveals the spectral characteristics of the audio signal from a neuromorphic perspective.

This analysis helps answer questions like:
- Which frequency bands are most prominent in the signal?
- Is the response broadband or narrowband?
- Are there distinct spectral peaks corresponding to specific sound features?

In [ ]:
def analyze_channel_response(spike_file, settings=None, top_n=10):
    """
    Analyze and visualize per-channel response characteristics.
    
    Parameters:
    -----------
    spike_file : SpikesFile
        PyNAVIS spike file object
    settings : MainSettings
        PyNAVIS settings
    top_n : int
        Number of top channels to highlight
        
    Returns:
    --------
    dict : Dictionary containing channel response metrics
    """
    timestamps = spike_file.timestamps
    addresses = spike_file.addresses
    
    if len(timestamps) == 0:
        print("No spikes to analyze!")
        return None
    
    ts_tick = settings.ts_tick if settings else 0.01
    total_duration_ms = (timestamps.max() - timestamps.min()) * ts_tick
    total_duration_s = total_duration_ms / 1000.0
    
    # Calculate per-channel statistics
    channel_spike_counts = np.bincount(addresses, minlength=64)
    channel_spike_rates = channel_spike_counts / total_duration_s  # spikes per second
    
    # Identify most responsive channels
    top_channels = np.argsort(channel_spike_rates)[-top_n:][::-1]
    
    # Calculate response concentration
    total_spikes = len(addresses)
    top_10_spikes = channel_spike_counts[top_channels[:10]].sum()
    concentration_ratio = top_10_spikes / total_spikes * 100
    
    # Calculate spectral spread
    weighted_mean = np.average(range(64), weights=channel_spike_counts)
    spectral_std = np.sqrt(np.average((np.arange(64) - weighted_mean)**2, weights=channel_spike_counts))
    
    print(f"\n{'='*60}")
    print(f"CHANNEL RESPONSE ANALYSIS")
    print(f"{'='*60}")
    print(f"  Total duration: {total_duration_s:.3f} seconds")
    print(f"  Total spikes: {total_spikes:,}")
    print(f"  Active channels: {np.sum(channel_spike_counts > 0)}/64")
    print(f"\n  Response concentration: {concentration_ratio:.1f}% in top 10 channels")
    print(f"  Spectral centroid: Channel {weighted_mean:.1f}")
    print(f"  Spectral spread: {spectral_std:.1f} channels")
    print(f"\n  Top {top_n} most responsive channels:")
    for rank, ch in enumerate(top_channels, 1):
        rate = channel_spike_rates[ch]
        count = channel_spike_counts[ch]
        percentage = count / total_spikes * 100
        print(f"    {rank:2d}. Channel {ch:2d}: {rate:7.1f} spikes/s ({count:6,} spikes, {percentage:5.2f}%)")
    print(f"{'='*60}")
    
    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot 1: Spike rate per channel (bar chart)
    ax1 = axes[0, 0]
    colors = ['red' if ch in top_channels else 'steelblue' for ch in range(64)]
    ax1.bar(range(64), channel_spike_rates, color=colors, edgecolor='black', linewidth=0.5)
    ax1.set_xlabel('Frequency Channel (High → Low)', fontsize=11)
    ax1.set_ylabel('Spike Rate (spikes/second)', fontsize=11)
    ax1.set_title('Channel Response Profile', fontsize=13, fontweight='bold')
    ax1.set_xlim(-1, 64)
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Plot 2: Cumulative response
    ax2 = axes[0, 1]
    sorted_rates = np.sort(channel_spike_rates)[::-1]
    cumulative_percent = np.cumsum(sorted_rates) / np.sum(sorted_rates) * 100
    ax2.plot(range(1, 65), cumulative_percent, 'b-', linewidth=2)
    ax2.axhline(y=80, color='red', linestyle='--', alpha=0.5, label='80% threshold')
    ax2.axhline(y=50, color='orange', linestyle='--', alpha=0.5, label='50% threshold')
    ax2.set_xlabel('Number of Top Channels', fontsize=11)
    ax2.set_ylabel('Cumulative Response (%)', fontsize=11)
    ax2.set_title('Cumulative Response Distribution', fontsize=13, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(0, 64)
    ax2.set_ylim(0, 105)
    
    # Plot 3: Spike count distribution (log scale)
    ax3 = axes[1, 0]
    non_zero_counts = channel_spike_counts[channel_spike_counts > 0]
    ax3.hist(non_zero_counts, bins=30, color='steelblue', edgecolor='black', alpha=0.7)
    ax3.set_xlabel('Spike Count', fontsize=11)
    ax3.set_ylabel('Number of Channels', fontsize=11)
    ax3.set_title('Distribution of Channel Activity Levels', fontsize=13, fontweight='bold')
    ax3.set_yscale('log')
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Frequency response heatmap view
    ax4 = axes[1, 1]
    # Reshape into matrix for better visualization
    response_matrix = channel_spike_rates.reshape(8, 8)
    im = ax4.imshow(response_matrix, cmap='hot', aspect='auto', interpolation='nearest')
    ax4.set_xlabel('Channel Group (×8)', fontsize=11)
    ax4.set_ylabel('Channel Subgroup', fontsize=11)
    ax4.set_title('Response Intensity Heatmap', fontsize=13, fontweight='bold')
    plt.colorbar(im, ax=ax4, label='Spike Rate (spikes/s)')
    
    plt.tight_layout()
    plt.show()
    
    return {
        'channel_spike_counts': channel_spike_counts,
        'channel_spike_rates': channel_spike_rates,
        'top_channels': top_channels,
        'spectral_centroid': weighted_mean,
        'spectral_spread': spectral_std,
        'concentration_ratio': concentration_ratio
    }

# Run channel response analysis
if len(spike_files) > 0:
    channel_info = analyze_channel_response(spike_files[0], settings=settings, top_n=10)
else:
    print("No spike data available for analysis!")

## 8. Exercise 1: Custom Monitoring Duration

**Objective:** Understand how monitoring duration affects data capture and analysis quality.

**Task:**  
Modify the monitoring parameters to capture events for different durations:
- Short duration: 0.1 seconds
- Medium duration: 1.0 seconds  
- Long duration: 2.0 seconds

For each duration:
1. Capture the spike data
2. Generate the four visualization types (spikegram, sonogram, histogram, average activity)
3. Compare the results and answer these questions:
   - How does the spike count scale with duration?
   - Which visualizations are most affected by shorter durations?
   - At what duration do you get a stable representation of the audio signal?

**Hint:** You only need to change the `DURATION` variable and re-run the monitoring section. All the plotting code can remain the same.

**Expected observations:**
- Shorter durations may miss transient events
- Longer durations provide more stable statistics but require more memory
- The histogram should stabilize as duration increases
- The sonogram quality improves with longer captures

In [ ]:
# Exercise 1: Try different monitoring durations
# TODO: Modify DURATION and run the monitoring + visualization cells

# Example code structure:
# DURATION = 0.1  # Try: 0.1, 1.0, 2.0 seconds
# spikes = okaer.monitor(inputs=INPUTS, duration=DURATION)
# ... create spike_files ...
# ... generate plots ...

# YOUR CODE HERE
# Experiment with different DURATION values and observe the differences

print("Exercise 1: Experiment with monitoring durations")
print("Suggested durations to try: 0.1s, 1.0s, 2.0s")
print("Observe how the visualizations change with duration")